 I more or less changed the order of things appearing in integrals, changed the computation of T and the adjoint W, because for the time derivative in the adjoint it takes step sizes smaller than the time step for having better approximations (the T(x,t+h)-T(x,t) ) and this works well for general functions but not for a solution T of the (SE). My approach: T(x,t) is basically defined as linear spline of the different T(x,t1), T(x,t2) ... The way to compute that is probably not good, but it works. This applied to nearly every computation we do for control, state and adjoint. The next thing we need to change is the gradient (Q-W does not work because both are iterators, so something like qq-ww for qq in Q and for ww in W would work, but I have a different idea). As said I will work on that tomorrow.

In [1]:
# import Pkg
# Pkg.add("FillArrays")

In [2]:
using Gridap
using Gridap.ODEs

# Modell und FE-Räume
model = CartesianDiscreteModel((0,1), 100)
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V = TestFESpace(model, reffe, conformity=:H1, dirichlet_tags="boundary")
U = TrialFESpace(V, [x -> 0.0])  # T=0 an beiden Enden

# Heizleistung Q(x,t) = 100 im mittleren Bereich
Q(x,t) = (x[1] > 0.4 && x[1] < 0.6) ? 100.0 : 0.0

# Schwache Form
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)
res(t, u, v) = ∫( ∂t(u)*v + ∇(v)⋅∇(u) - Q(x,t)*v )dΩ
jac(t, u, du, v) = ∫( ∇(v)⋅∇(du) )dΩ

# Zeitintegration
op = TransientFEOperator(res, jac, U, V)
Δt = 0.01
solver = ThetaMethod(LUSolver(), Δt, 0.5)  # Crank-Nicolson
uh0 = interpolate_everywhere(0.0, U)
uh_t = solve(solver, op, 0.0, 1.0, uh0)

# Ergebnisse speichern
writevtk(Ω, "results", cellfields=["uh" => uh_t(end)])

AssertionError: AssertionError: Incorrect number of dirichlet tags provided

# Experts in Teams

## version 1:
$\begin{align}
&\min E(Q,T)=\int_{t_1}^{t_F}\int_\Omega p(t)Q(x,t)\,\mathrm{d}x\mathrm dt+\frac\gamma2\int_\Omega |T(x,t_F)-T_\text{fin}(x)|^2\mathrm dx \\
&\begin{cases}
\rho c\dot T(x,t)&=\nabla\cdot\left(k(x)\nabla T(x,t)\right)+Q(x,t) && \text{in}\ \Omega\times(0,t_F) \\
-k(x)\partial_\nu T(x,t)&=h(x)\left(T(x,t)-T_\text{out}(x,t)\right) && \text{on}\ \partial\Omega\times[0,t_F]\\
T(x,0)=T_\text{ini}(x)&
\end{cases}\tag{SE}\\
& Q\in U_\text{ad}=\{Q\in L^2(\Omega\times(0,t_F))\,|\,0\leq Q(x,t)\leq Q_\text{max}(x)\ \text{a.e.}\}\\
&\text{optional}:\ T(x,t_F)\geq T_\text{fin}(x) (\text{optional if second part of E should be strict})
\end{align}$
where
$T(x,t)$ temperature in K (Henning: I will use °C, but written K is better), $Q(x,t)$ heating output in $\frac W {m^3}$, $p(t)$ price, $\rho$ density in $\frac{kg}{m^3}$, $c$ heat capacity in $\frac{J}{kg\cdot K}$, $k(x)$ heat transfer coefficient in $\frac W{m\cdot K}$, $h(x)$ convective heat transfer coefficient in $\frac W{m^2K}$, $t_1$ morning start time

Remember that $\partial_\nu T$ is the outer normal derivative with respect to $x$ and any initial data (or end data) would have to satisfy the PDE.

In order to solve the (SE) we will use a more basic method, in which we seperate between the spacial and temporal derivatives and first solve the PDE in space using approximations of the time derivatives and then the ODE in time or we try to decouple both variables as much as possible to get a linear system in block structure. This is done by Gridap automatically. Thus, we need the weak formulation only in space (so a semi-weak formulation of the whole system). Here we will give it in its weak form remarking that every time integral of the (WF) for the (SWF) would be replaced by "for all $t\in(0,t_F)$", because computing the derivative of the energy functional is easier this way without loosing track of the $t$, and get with the test space $H^1_{\Gamma}(\Omega\times(0,t_F))$ where $\Gamma=\Omega\times\{0\}$
$\begin{align}
\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot T(x,t)\phi(x,t)\,\mathrm dx\mathrm dt=\int\limits_0^{t_F}\int\limits_\Omega k(x)\nabla T(x,t)\cdot\nabla \phi(x,t)+Q(x,t)\phi(x,t)\,\mathrm dx\mathrm dt+\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\left(T(y,t)-T_\text{out}(y)\right)\phi(y,t)\,\mathrm dy\mathrm dt \tag{WF}
\end{align}$
giving us
$\begin{align}
a_\text{SE}(T,\phi)=\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot T(x,t)\phi(x,t)-k(x)\nabla T(x,t)\cdot\nabla \phi(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)T(y,t)\phi(y,t)\,\mathrm dy\mathrm dt,\qquad\qquad
l_\text{SE}(\phi)=\int\limits_0^{t_F}\int\limits_\Omega Q(x,t)\phi(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)T_\text{out}(y)\phi(y,t)\,\mathrm dy\mathrm dt
\end{align}$
Henning: remember that each boundary integral could be rewritten with the Trace operator, might be helpful for the next part. When writing the (SWF) the open interval for $∀ t\in(0,t_F)$ is important.

From the optimisation functional and the (WF) we get the Lagrangian as
$\begin{align}
L(T,q,W)=\int\limits_{t_1}^{t_F}\int\limits_\Omega p(t)Q(x,t)\,\mathrm{d}x\mathrm dt+\frac\gamma2\int\limits_\Omega |T(x,t_F)-T_\text{fin}(x)|^2\mathrm dx+\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot T(x,t)W(x,t)-k(x)\nabla T(x,t)\cdot\nabla W(x,t)-Q(x,t)W(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\left(T(y,t)-T_\text{out}(y)\right)W(y,t)\,\mathrm dy\mathrm dt
\end{align}$
which gives us the weak formulation of the adjoint (AE) as (notice that the second term contains $\dot\psi$ which in the (SWF) would be zero and thus needs to be solved differently: in that case we are actually only looking for the derivative of the $x$ dendent $T$, so that the derivative $\partial_{T(x)}T(x,t)$ does not give 1 but a time dependent function)
$\begin{align}
L_T(T,Q,W)\psi=\gamma\int\limits_\Omega \left(T(x,t_F)-T_\text{fin}(x)\right)\psi(x,t_F)\mathrm dx+\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot \psi(x,t)W(x,t)-k(x)\nabla \psi(x,t)\cdot\nabla W(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\psi(y,t)W(y,t)\,\mathrm dy\mathrm dt
\end{align}$
By Green for any $t\in(0,t_F)$ it is
$\begin{align}
\int\limits_\Omega -k(x)\nabla\psi(x,t)\cdot\nabla W(x,t)\,\mathrm dx=\int\limits_\Omega \nabla\cdot\left(k(x)\nabla W(x,t)\right)\psi(x,t)\,\mathrm dx-\int\limits_{\partial\Omega} k(y)\partial_\nu W(y,t)\psi(y,t)\,\mathrm dy
\end{align}$
and from Green for the $t$ variable (integration by parts) we get
$\begin{align}
\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot \psi(x,t)W(x,t)\,\mathrm dx\mathrm dt=-\int\limits_0^{t_F}\int\limits_\Omega \rho c\psi(x,t)\dot W(x,t)\,\mathrm dx\mathrm dt+\left[\int\limits_\Omega \rho c\psi(x,t)W(x,t)\,\mathrm dx\right]_0^{t_F}=-\int\limits_0^{t_F}\int\limits_\Omega \rho c\psi(x,t)\dot W(x,t)\,\mathrm dx\mathrm dt+\int\limits_\Omega \rho c\psi(x,t_F)W(x,t_F)\,\mathrm dx
\end{align}$
when assuming $W(x,0)=0$ a.e. (because $W$ has to be able to operate as test function for (SE)) and thus $\psi\in H^1_\Gamma(\Omega\times(0,t_F))$. With these it is
$\begin{align}
L_T(T,Q,W)\psi=\gamma\int\limits_\Omega \left(T(x,t_F)-T_\text{fin}(x)\right)\psi(x,t)\mathrm dx-\int\limits_0^{t_F}\int\limits_\Omega \rho c\psi(x,t)\dot W(x,t)-\nabla\cdot\left(k(x)\nabla W(x,t)\right)\psi(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\psi(y,t)W(y,t)+k(y)\partial_\nu W(y,t)\psi(y,t)\,\mathrm dy\mathrm dt+\int\limits_\Omega \rho c\psi(x,t_F)W(x,t_F)\,\mathrm dx
\end{align}$
giving the strong version
$\begin{align}
\begin{cases}
\rho c\dot W(x,t)&=\nabla\cdot\left(k(x)\nabla W(x,t)\right)&&\text{in}\ \Omega\times(0,t_F) \\
-k(y)\partial_\nu W(y,t)&=h(y)W(y,t)&&\text{on}\ \partial\Omega\times[0,t_F] \\
W(x,t_F)&=\frac{\gamma}{\rho c}\left(T(x,t_F)-T_\text{fin}(x)\right)&&\text{in}\ \Omega
\end{cases}\tag{AE}
\end{align}$
Thus, we can see that the (AE) has to be solved backwards in time. The weak version is
$\begin{align}
\int\limits_0^{t_F}\int\limits_\Omega \rho c\psi(x,t)\dot W(x,t)+k(x)\nabla \psi(x,t)\cdot\nabla W(x,t)\,\mathrm dx\mathrm dt=-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\psi(y,t)W(y,t)\,\mathrm dy\mathrm dt \tag{WAE}
\end{align}$
For the (AE) we have
$\begin{align}
a_\text{AE}(W,\psi)=\int\limits_0^{t_F}\int\limits_\Omega \rho c\psi(x,t)\dot W(x,t)+k(x)\nabla \psi(x,t)\cdot\nabla W(x,t)\,\mathrm dx\mathrm dt+\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\psi(y,t)W(y,t)\,\mathrm dy\mathrm dt\qquad\qquad l_\text{AE}(\psi)=0
\end{align}$
Henning: Actually, we get the (AE) by the variation principle and not by differentiation, which does not fully make sense here.

Now we introduce the solution operator $S$, which for a given $Q$ gives the solution $T$ of the (SE), so $T=S(Q)$. With this we can introduce the reduced energy functional
$\begin{align}
e(Q)=\int_{t_1}^{t_F}\int_\Omega p(t)Q(x,t)\,\mathrm{d}x\mathrm dt+\frac\gamma2\int_\Omega |S(Q)(x,t_F)-T_\text{fin}(x)|^2\mathrm dx
\end{align}$
In order to find the gradient of $e$ we differentiate it and find by noting that $S$ is linear and therefore has derivative $S'(Q)h=Sh$
$\begin{align}
e'(Q)h&=\int_{t_1}^{t_F}\int_\Omega p(t)h(x,t)\,\mathrm{d}x\mathrm dt+\gamma\int_\Omega (S(Q)(x,t_F)-T_\text{fin}(x))Sh\mathrm dx\\
&=\langle p|h\rangle+\langle S(Q)-T_\text{fin} | Sh\rangle|_{t=t_F}=\langle p|h\rangle+\langle S^*(S(Q)-T_\text{fin}) | h\rangle|_{t=t_F} \equiv p+W
\end{align}$
where $S^*$ is the solution operator for (AE) and $W$ is the solution ........ 


## version 2:
$\begin{align}
&\min E(Q,T)=\int_{t_1}^{t_F}\int_\Omega (p(t)Q(x,t))^2\,\mathrm{d}x\mathrm dt+\frac\gamma2\int_\Omega |T(x,t_F)-T_\text{fin}(x)|^2\mathrm dx \\
&\begin{cases}
\rho c\dot T(x,t)&=\nabla\cdot\left(k(x)\nabla T(x,t)\right)+Q(x,t) && \text{in}\ \Omega\times(0,t_F) \\
-k(x)\partial_\nu T(x,t)&=h(x)\left(T(x,t)-T_\text{out}(x,t)\right) && \text{on}\ \partial\Omega\times[0,t_F]\\
T(x,0)=T_\text{ini}(x)&
\end{cases}\tag{SE}\\
& Q\in U_\text{ad}=\{Q\in L^2(\Omega\times(0,t_F))\,|\,0\leq Q(x,t)\leq Q_\text{max}(x)\ \text{a.e.}\}\\
&\text{optional}:\ T(x,t_F)\geq T_\text{fin}(x) (\text{optional if second part of E should be strict})
\end{align}$
In this case everything stays the same but the Lagrangian is
$\begin{align}
L(T,q,W)=\int\limits_{t_1}^{t_F}\int\limits_\Omega (p(t)Q(x,t))^2\,\mathrm{d}x\mathrm dt+\frac\gamma2\int\limits_\Omega |T(x,t_F)-T_\text{fin}(x)|^2\mathrm dx+\int\limits_0^{t_F}\int\limits_\Omega \rho c\dot T(x,t)W(x,t)-k(x)\nabla T(x,t)\cdot\nabla W(x,t)-Q(x,t)W(x,t)\,\mathrm dx\mathrm dt-\int\limits_0^{t_F}\int\limits_{\partial\Omega} h(y)\left(T(y,t)-T_\text{out}(y)\right)W(y,t)\,\mathrm dy\mathrm dt
\end{align}$

Now we introduce the solution operator $S$, which for a given $Q$ gives the solution $T$ of the (SE), so $T=S(Q)$. With this we can introduce the reduced energy functional
$\begin{align}
e(Q)=\int_{t_1}^{t_F}\int_\Omega (p(t)Q(x,t))^2\,\mathrm{d}x\mathrm dt+\frac\gamma2\int_\Omega |S(Q)(x,t_F)-T_\text{fin}(x)|^2\mathrm dx
\end{align}$
In order to find the gradient of $e$ we differentiate it and find by noting that $S$ is linear and therefore has derivative $S'(Q)h=Sh$
$\begin{align}
e'(Q)h&=\int_{t_1}^{t_F}\int_\Omega 2p(t)Q(x,t)h(x,t)\,\mathrm{d}x\mathrm dt+\gamma\int_\Omega (S(Q)(x,t_F)-T_\text{fin}(x))Sh\mathrm dx\\
&=\langle 2pQ|h\rangle+\langle S(Q)-T_\text{fin} | Sh\rangle|_{t=t_F}=2\langle pQ|h\rangle+\langle S^*(S(Q)-T_\text{fin}) | h\rangle|_{t=t_F} \equiv 2pQ+W
\end{align}$



Thus, without box constraints from the condition $e'(Q)=0$ we would get $Q=-\frac{1}{2p}W$ and can use the projection of that onto the box for our case (see bang bang).


### Other:
Remember the Courant-Friedrichs-Lewy (CFL) condition: If the numerical solution is screwed up it might be because space step length / time step length is too small. Honestly, I do not know the specific condition for that problem, it might even be reversed here, but this should be considered.

huhu

Things to do:
the solver for (AE) 

In [3]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

function GradientDescent(;solveSE, solveAE, spaces, dΩ, dΓ=nothing, J, ∇f, iter_max=1000, tol=1e-3, P=x->x, u0=nothing, w=nothing, s_min=nothing, sminargs=nothing, armijoparas=(ρ=1/2, α_0=1, α_min=1/2^5, σ=1e-4), saveall::Bool=false)
	#=
	Solves any CP with possibly box bounds
	Input: (all are named arguments - oder does not matter, but they have to be called by name)
	  solveSE   - a method to solve the SE, Input: u (rhs), Trialspace, Testspace. optional: dΩ, dΓ, cache from earlier execution, y_dof pre-solution in vector-form to write on. Output: solution, cache
	  solve AE  - a method to solve the AE, Input: y (rhs), Trialspace, Testspace. optional: dΩ, dΓ, cache from earlier execution, p_dof pre-solution in vector-form to write on. Output: solution, cache
	  spaces    - tuple of (trial space, test space, FE space)
	  dΩ        - measure on Ω
	  dΓ        - measure on ∂Ω=Γ (optional)
	  J         - functional to minimise (only needed for early breaking out of loop)
	  ∇f        - functional to compute gradient of reduced cost f
	  iter_max  - number of maximum iterations (default 2000)
	  tol       - tolerance for early break out of loop (default 10^(-3))
	  P         - Projection onto box. P is assumed to have the form u↦P(u) and should be able to handle an FEFunction as input (default without) and give such as output
	  u0        - start function for u (optional, default without)
	  w         - boundary term (default without)
	  s_min     - function to compute exact step size (default without - then Armijo rule will be used). Input is supposed to be y,u,w,p (from algorithm) and sminargs will be unloaded
	  sminargs  - additional arguments for s_min that can be given
	  armijoparas   - parameters for Armijo rule. defaults are given and can individually be changed (named arguments)
	=#
	Trialspace, Testspace, Qspace = spaces                                  # Extract spaces

	q = FEFunction(Qspace, rand(Float64, num_free_dofs(Qspace)) |> P)       # Initialize u with some random values and apply projection

	y, cacheSE, A_SE = solveSE(q,Trialspace,Testspace,w=w,dΩ=dΩ,dΓ=dΓ)  # initial SE solve
	#y = FEFunction(Trialspace, y_dof)

	p, cacheAE, A_AE = solveAE(y,q,Trialspace,Testspace,dΩ=dΩ,dΓ=dΓ)      # initial AE solve
	#p = FEFunction(Testspace, p_dof)
	
	cost = J(y, q)
	fgrad =  ∇f(q, p, y)														# Compute initial gradient
	L2fgrad0 = √(∑(∫(fgrad⋅fgrad)*dΩ))                                       # Compute norm of initial gradient

	if saveall
		qs=[q]                     # save the solutions - only if really necessary
		ys=[y]
		ps=[p]
		costs=[cost]
	else
		qs,ys,ps=[],[],[]
	end
	for k=1:iter_max
		if √(∑(∫(fgrad⋅fgrad)*dΩ)) < tol*L2fgrad0                           # loop break condition - better ideas are appreciated
			break
		end
		q_new = y_new = cost_new = nothing
		if s_min!=nothing                                                   # if method for exact step size is defined use that
			s = s_min(y,q,w,p,sminargs=(cacheSE,A_SE,sminargs))
			q_new = interpolate_everywhere(q - s*fgrad,Qspace) |> P			# in most cases interpolate instead of interpolate_everywhere works as well
			y_dof, cacheSE = solveSE(q_new,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ,cache=cacheSE,A=A_SE,y_dof=y_dof)
			y_new = FEFunction(Trialspace, y_dof)
			cost_new = J(y,q)
		else                                                                # else use Armijo rule - if projection is used, you should consider smaller step size than default
			ρ, α_0, α_min, σ = armijoparas
			cost_new = cost
			α = α_0
			while α > α_min
				q_new = interpolate_everywhere(q - α*fgrad, Qspace) |> P    # Compute tentative new control function defined by current line search parameter

				y_new, cacheSE = solveSE(q_new,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ,cache=cacheSE,A=A_SE,y_dof=y_dof)
				#y_new = FEFunction(Trialspace, y_dof)

				cost_new = J(y_new, q_new)                                  # Compare decrease in functional and accept if sufficient
				if cost_new < cost - σ*α*∑(∫(fgrad⋅fgrad)*dΩ)
					break
				else
					α *= ρ
				end
			end
		end# here you could implement other methods
		q = q_new
		y = y_new
		cost = cost_new
		
		if saveall
			push!(qs,FEFunction(Qspace, get_free_dof_values(q)))
			push!(ys,y)
			push!(costs,cost)
		end
		p, cacheAE = solveAE(y,q,Trialspace,Testspace;dΩ=dΩ,dΓ=dΓ,cache=cacheAE,A=A_SE,p_dof=p_dof)
		#p = FEFunction(Testspace, p_dof)

		fgrad = ∇f(q, p, y)
		push!(ps,p)
	end
	return saveall ? (ys,qs,ps,costs) : (y,q,p,cost)                    	# give back either all saved variables or only end result
end

function draw(ysol)
	fig, _ , plt = CairoMakie.plot(Ω, ysol, colormap=:plasma)               # plot of last state (numerical solution)
	CairoMakie.wireframe!(Ω, color=:black, linewidth=1)                        # add triangulation
	CairoMakie.Colorbar(fig[1,2], plt)                                         # add color bar
	display(fig)															  # display the plot
end


domain = (0,1,0,1)
partition = (20,20)
model = CartesianDiscreteModel(domain,partition) |> simplexify
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1) ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions

Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                    # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)                                              # make the measure dΩ

ρ=1.225
c=1020.0
k=15.0
h=0.7
Tout(x,t)=5.0
Q(x,t)=0.0
Tini(x)=0.0
t0=0.0
tF=10.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.05

γ = 1
function E(T,P)
	E=0.0
	for (TT,t) in T
		tmp=TT-20.0
		E+=Δt*∑(∫(tmp*tmp)*dΩ)
	end
	for (PP,t) in P
		E+=Δt*γ*∑(∫(PP*PP)*dΩ)
	end
	return E/2.0
end
P(a,b,z) = min(max(a,z),b)
# a=0.0
# b=1.0
P(z)=map(x->P(0.0,1.0,x),z)
P(z::Gridap.FESpaces.SingleFieldFEFunction)=FEFunction(Uspace,map(x->P(a,b,x),get_free_dof_values(z)))

# ∇f(q, p, y) = (γ*q - y*p)                                                # gradient of reduced cost
∇f(Q, p, W) = (2*p*Q + W)                                                # gradient of reduced cost



ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)


a_AE_tconst(t, dtW, ψ) = ∫(ρ*c*dtW*ψ)dΩ
a_AE_tnonconst(t, W, ψ) = ∫(+k * ∇(T) ⋅ ∇(ψ) + tr(h*T*ψ))dΩ
l_AE(t, ψ) = γ*∫(((x->T(x,t))-Tfin)*ψ - tr(h*(x->Tout(x,t))*ψ))dΩ	
op_AE = TransientLinearFEOperator((a_AE_tconst, a_AE_tnonconst), l_AE, Trialspace, Testspace, constant_forms=(true, false))


function SEsolver(Q,Trialspace,Testspace;w=nothing,dΩ,dΓ=nothing,cache=nothing,A=nothing,y_dof=fill(0.0, num_free_dofs(Testspace)))
	a_SE_tconst(t, dtT, ϕ) = ∫(ρ*c*dtT*ϕ)dΩ
	a_SE_tnonconst(t, T, ϕ) = ∫(-k * ∇(T) ⋅ ∇(ϕ) - tr(h*T*ϕ))dΩ
	l_SE(t, ϕ) = ∫((x->Q(x,t))*ϕ - tr(h*(x->Tout(x,t))*ϕ))dΩ	
	op_SE = TransientLinearFEOperator((a_SE_tconst, a_SE_tnonconst), l_SE, Trialspace, Testspace, constant_forms=(true, false))
	# tableau = :SDIRK_2_2
	# solver_rk = RungeKutta(ls, ls, Δt, tableau)

	y_dof = solve(solver, op_SE, t0, tF, TIni)
	return y_dof, 0.0,0.0
end
function AEsolver(T,Q,Trialspace,Testspace;dΩ,dΓ=nothing,cache=nothing,A=nothing,p_dof=fill(0.0, num_free_dofs(Testspace)))
	a_AE_tconst(t, dtW, ψ) = ∫(ρ*c*dtW*ψ)dΩ
	a_AE_tnonconst(t, W, ψ) = ∫(+k * ∇(W) ⋅ ∇(ψ) + tr(h*W*ψ))dΩ
	l_AE(t, ψ) = γ*∫(((x->T(x,t))-Tfin)*ψ - tr(h*(x->Tout(x,t))*ψ))dΩ	
	op_AE = TransientLinearFEOperator((a_AE_tconst, a_AE_tnonconst), l_AE, Trialspace, Testspace, constant_forms=(true, false))
	# tableau = :SDIRK_2_2
	# solver_rk = RungeKutta(ls, ls, Δt, tableau)

	y_dof = solve(solver, op_AE, t0, tF, TIni)
	return y_dof, 0.0,0.0
end


## Lastly we want to use the remark from the lecture and construct a solver function for the optimal step length. (see my extra remark)
function s_min(y,q,w,p;sminargs)
	cache,A=sminargs
	v=interpolate(-y*p+γ*q, Uspace)                                      # v=p+γ*u, since all uses of v are in norms we can ignore the minus
	Sv,_ = SEsolver(v,Trialspace,Testspace;dΩ=dΩ,cache=cache,A=A)
	Sv = FEFunction(Trialspace,Sv)
	L2NormSquaredOfv = ∑(∫(v*v)*dΩ)                                   # ||v||^2
	return L2NormSquaredOfv/(∑(∫(Sv*Sv)*dΩ)+γ*L2NormSquaredOfv)
end

(ys,qs,ps,costs) = GradientDescent(;solveSE=SEsolver, solveAE=AEsolver, spaces=(Trialspace, Testspace, Uspace), dΩ=dΩ, J=E, ∇f=∇f, P=P, sminargs=nothing, saveall=true, tol=1e-13, iter_max=25000, armijoparas=(ρ=1/2, α_0=100, α_min=0, σ=1e-4))

MethodError: MethodError: no method matching (::SingleFieldFEFunction{GenericCellField{ReferenceDomain}})(::VectorValue{2, Float64}, ::Float64)
The object of type `SingleFieldFEFunction{GenericCellField{ReferenceDomain}}` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

Closest candidates are:
  (::CellField)(::Any)
   @ Gridap C:\Users\henni\.julia\packages\Gridap\Ce4E8\src\CellData\CellFields.jl:388


In [1]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils

domain = (0,1,0,1)
partition = (20,20)
model = CartesianDiscreteModel(domain,partition) |> simplexify
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1) ###### conformity correct?
Trialspace = TransientTrialFESpace(Testspace)                                # maybe add a function for/if Dirichlet conditions

Uspace = FESpace(model, reffe, conformity=:H1)

degree = 2*order                                                    # degree of the method used for approximating integrals over Ω
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)                                              # make the measure dΩ
Γ = BoundaryTriangulation(model)                                    # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)                                              # measure on Γ

ρ(x)=1.225
c(x)=1020.0
k(x)=15.0
h(x)=0.7
Tout(x,t)=5.0
Q(x,t)=0.0
Tini(x)=0.0
t0=0.0
tF=10.0
TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))
Δt = 0.05

γ = 1
function E(T,P)
	E=0.0
	for (TT,t) in T
		tmp=TT-20.0
		E+=Δt*∑(∫(tmp*tmp)*dΩ)
	end
	for (PP,t) in P
		E+=Δt*γ*∑(∫(PP*PP)*dΩ)
	end
	return E/2.0
end
P(a,b,z) = min(max(a,z),b)
# a=0.0
# b=1.0
P(z)=map(x->P(0.0,1.0,x),z)
P(z::Gridap.FESpaces.SingleFieldFEFunction)=FEFunction(Uspace,map(x->P(a,b,x),get_free_dof_values(z)))

∇f(Q,T,W) = (Q .- W)                                                # gradient of reduced cost

ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

function find(y,s)
	tsave,ysave=t0,nothing
	for (t,yy) in y
		if t==0
			ysave=yy
		end
		if t≈s
			return yy
		elseif s+Δt >= t
			return interpolate_everywhere(((s-tsave)*ysave+(t-s)*yy)/Δt, Uspace(s))
		end
		tsave,ysave = t,yy
	end
end

#q = FEFunction(Uspace, rand(Float64, num_free_dofs(Uspace)) |> P)
Q(t)=x->Q(x,t)
Tout(t)=x->Tout(x,t)

a_SE_tconst(t, dtT, ϕ) = ∫(c*dtT*ϕ*ρ)dΩ
a_SE_tnonconst(t, T, ϕ) = (-1)*∫(k * ∇(T) ⋅ ∇(ϕ) + tr(h*T*ϕ))dΩ
l_SE(t, ϕ) = ∫(Q(t) * ϕ)dΩ - ∫(Tout(t) * ϕ * h)dΓ
op_SE = TransientLinearFEOperator((a_SE_tconst, a_SE_tnonconst), l_SE, Trialspace, Testspace, constant_forms=(true, false))
T = solve(solver, op_SE, t0, tF, TIni)

Tsave=[(t0,TIni),collect((t,TT) for (t,TT) in T)...]
T_temp(t)=find(Tsave,t)

a_AE_tconst(t, dtW, ψ) = ∫(c*dtW*ψ*ρ)dΩ
a_AE_tnonconst(t, W, ψ) = ∫(k * ∇(W) ⋅ ∇(ψ))dΩ + ∫(h*W*ψ)dΓ
l_AE(t, ψ) = ∫(T_temp(t)*ψ-Tfin*ψ)dΩ - ∫(Tout(t)*ψ*h)dΓ	
op_AE = TransientLinearFEOperator((a_AE_tconst, a_AE_tnonconst), l_AE, Trialspace, Testspace, constant_forms=(true, false))
W = solve(ThetaMethod(ls, 2*Δt, θ), op_AE, t0, tF, TIni) # 2*Δt because the theta method want to evaluate the function at t+1/2*Δt

W_end=interpolate_everywhere(γ*(T_temp(tF)-Tfin)/c/ρ, Uspace(tF))
Wsave=[collect((t,WW) for (t,WW) in W)... ; (tF,W_end)]

cost=E(Tsave,Wsave)
gradient=∇f(Q,Tsave,Wsave)


MethodError: MethodError: no method matching -(::typeof(Q), ::Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}})
The function `-` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  -(!Matched::ChainRulesCore.NoTangent, ::Any)
   @ ChainRulesCore C:\Users\henni\.julia\packages\ChainRulesCore\U6wNx\src\tangent_arithmetic.jl:61
  -(::Any, !Matched::ChainRulesCore.NotImplemented)
   @ ChainRulesCore C:\Users\henni\.julia\packages\ChainRulesCore\U6wNx\src\tangent_arithmetic.jl:50
  -(!Matched::ChainRulesCore.NotImplemented, ::Any)
   @ ChainRulesCore C:\Users\henni\.julia\packages\ChainRulesCore\U6wNx\src\tangent_arithmetic.jl:49
  ...


In [5]:
for (t,w) in W
	println(get_free_dof_values(w))
end

UndefVarError: UndefVarError: `W` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
function GradientDescent(;solveSE, solveAE, spaces, dΩ, dΓ=nothing, J, ∇f, iter_max=1000, tol=1e-3, P=x->x, u0=nothing, w=nothing, s_min=nothing, sminargs=nothing, armijoparas=(ρ=1/2, α_0=1, α_min=1/2^5, σ=1e-4), saveall::Bool=false)
	Trialspace, Testspace, Qspace = spaces                                  # Extract spaces

	q = FEFunction(Qspace, rand(Float64, num_free_dofs(Qspace)) |> P)       # Initialize u with some random values and apply projection

	y, cacheSE, A_SE = solveSE(q,Trialspace,Testspace,w=w,dΩ=dΩ,dΓ=dΓ)  # initial SE solve
	#y = FEFunction(Trialspace, y_dof)

	p, cacheAE, A_AE = solveAE(y,q,Trialspace,Testspace,dΩ=dΩ,dΓ=dΓ)      # initial AE solve
	#p = FEFunction(Testspace, p_dof)
	
	cost = J(y, q)
	fgrad =  ∇f(q, p, y)														# Compute initial gradient
	L2fgrad0 = √(∑(∫(fgrad⋅fgrad)*dΩ))                                       # Compute norm of initial gradient

	if saveall
		qs=[q]                     # save the solutions - only if really necessary
		ys=[y]
		ps=[p]
		costs=[cost]
	else
		qs,ys,ps=[],[],[]
	end
	for k=1:iter_max
		if √(∑(∫(fgrad⋅fgrad)*dΩ)) < tol*L2fgrad0                           # loop break condition - better ideas are appreciated
			break
		end
		q_new = y_new = cost_new = nothing
		if s_min!=nothing                                                   # if method for exact step size is defined use that
			s = s_min(y,q,w,p,sminargs=(cacheSE,A_SE,sminargs))
			q_new = interpolate_everywhere(q - s*fgrad,Qspace) |> P			# in most cases interpolate instead of interpolate_everywhere works as well
			y_dof, cacheSE = solveSE(q_new,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ,cache=cacheSE,A=A_SE,y_dof=y_dof)
			y_new = FEFunction(Trialspace, y_dof)
			cost_new = J(y,q)
		else                                                                # else use Armijo rule - if projection is used, you should consider smaller step size than default
			ρ, α_0, α_min, σ = armijoparas
			cost_new = cost
			α = α_0
			while α > α_min
				q_new = interpolate_everywhere(q - α*fgrad, Qspace) |> P    # Compute tentative new control function defined by current line search parameter

				y_new, cacheSE = solveSE(q_new,Trialspace,Testspace;w=w,dΩ=dΩ,dΓ=dΓ,cache=cacheSE,A=A_SE,y_dof=y_dof)
				#y_new = FEFunction(Trialspace, y_dof)

				cost_new = J(y_new, q_new)                                  # Compare decrease in functional and accept if sufficient
				if cost_new < cost - σ*α*∑(∫(fgrad⋅fgrad)*dΩ)
					break
				else
					α *= ρ
				end
			end
		end# here you could implement other methods
		q = q_new
		y = y_new
		cost = cost_new
		
		if saveall
			push!(qs,FEFunction(Qspace, get_free_dof_values(q)))
			push!(ys,y)
			push!(costs,cost)
		end
		p, cacheAE = solveAE(y,q,Trialspace,Testspace;dΩ=dΩ,dΓ=dΓ,cache=cacheAE,A=A_SE,p_dof=p_dof)
		#p = FEFunction(Testspace, p_dof)

		fgrad = ∇f(q, p, y)
		push!(ps,p)
	end
	return saveall ? (ys,qs,ps,costs) : (y,q,p,cost)                    	# give back either all saved variables or only end result
end

GradientDescent (generic function with 1 method)